## Following section tests dynamic normalisation, rgb and grayscale

In [10]:
from PIL import Image
import sys 
import os
# import torch, torchvision
import numpy as np
import time
sys.path.append(os.path.abspath("kx59/pytorch-CycleGAN-and-pix2pix/data"))
from image_folder import default_loader
from image_folder import make_dataset, dynamic_normalisation

In [11]:
trainA = '/home/arid/kx59/pytorch-CycleGAN-and-pix2pix/datasets/datasetSS1to4/trainA'
trainB = '/home/arid/kx59/pytorch-CycleGAN-and-pix2pix/datasets/datasetSS1to4/trainB'


In [12]:
trainAimgs = make_dataset(trainA)
trainBimgs = make_dataset(trainB)
imgs = trainAimgs+trainBimgs

In [16]:
stats = dynamic_normalisation(trainAimgs[:10],trainBimgs[:10])

Calculating normalisation stats
Total time for dyno calculation taken: 0.100009 seconds
Time taken per image for dyno calculation: 0.005000 seconds


In [8]:
statsGray = dynamic_normalisation(trainAimgs[:10],trainBimgs[:10],grayscale=True)

Calculating normalisation stats
Total time for dyno calculation taken: 0.059114 seconds
Time taken per image for dyno calculation: 0.002956 seconds


In [9]:
print(stats)
print(statsGray)

{'mean': [0.35970781281260583, 0.4653655055772522, 0.49396675467059087], 'std': [0.17775270614878358, 0.16966115086553138, 0.16806500877316496]}
{'mean': [0.4370183112307423], 'std': [0.16660754845647935]}


In [22]:
a = [1,2]
a.append((3,3))
print(a)

[1, 2, (3, 3)]


In [2]:
92*200/60/60

5.111111111111112

In [8]:
def calculate_image_stats(paths, gray=False):
    images = []
    start_time = time.time()
    for path in paths:
        with Image.open(path) as image:
            if gray:
                image = image.convert('L') # convert to grayscale
            images.append(np.array(image)/255)
                

    mean = np.mean(images, axis=(0, 1, 2), dtype=np.float64) # calculate mean of all pixels. Will return individual stats for each channel
    std = np.std(images, axis=(0, 1, 2), dtype=np.float64) # calculate standard deviation of all pixels. Will return individual stats for each channel
    end_time = time.time()-start_time
    print(f'Time taken: {end_time} seconds')
    print(f'Time taken per image: {(end_time/len(paths))} seconds')
    del images
    return {'mean':mean, 'std':std}

In [25]:
def shifted_data_variance(paths, gray=False):
    """
    Here we choose K to be an estimate of the mean per channel
    We shift every pixel in a given channel by the associated K. 
    This ensures no cancellation will occur due to the subtraction 
    during the 'var' assignment statement, otherwise a potential 
    cancellation could ruin the natural precision. This does not 
    change the variance, and thus standard deviation, since Var(X) = Var(X-K)
    """
    Sums = []
    SSq = []
    K = (0.412 if gray else np.array([0.338, 0.439, 0.463]).reshape(1,1,3))
    totalPix = 0 # counter for total number of pixels in dataset
    # if gray:
    #     K = np.array(0.412)
    # else:
    #     K = np.array([0.338, 0.439, 0.463])
    start_time = time.time()
    for path in paths:
        with Image.open(path) as image:
            if gray:
                image = image.convert('L') # convert to grayscale
            else:
                image = image.convert('RGB')
            image = np.array(image)/255
            totalPix += image.shape[0]*image.shape[1] # increment by the image height*width, i.e. number of pixels per channel
            
            shift = image-K 
            
            Sums.append(np.sum(shift,axis=(0,1))) # sum along the height and width (indices 0 and 1) but not along the channels
            SSq.append(np.sum(shift**2,axis=(0,1))) # sum along the height and width (indices 0 and 1) but not along the channels
            
    TotalSum = np.sum(Sums,axis=0) # sum the appended items along the axis of appending
    TotalSSq = np.sum(SSq,axis=0)
    
    mean = (K + TotalSum/totalPix).flatten().tolist()
    var = (TotalSSq-TotalSum**2/totalPix)/(totalPix-1) 
    # use (totalPix) instead of (totalPix-1) if want to compute the exact variance of the given data
    # use (totalPix-1) if data are samples of a larger population
    std = np.sqrt(var).flatten().tolist()
    
    end_time = time.time()-start_time
    print('Total time for dyno calculation taken: %.6f seconds' % end_time)
    print('Time taken per image for dyno calculation: %.6f seconds' % (end_time/len(paths)))
    del Sums, SSq
    if gray:
        mean.append(mean[0])
        mean.append(mean[0])
        std.append(std[0])
        std.append(std[0])
    return {'mean':mean,'std':std}

In [26]:
stats = shifted_data_variance(trainAimgs[:10]+trainBimgs[:10], gray=False)
print('Shifted algorithm stats for rgb:')
print(stats)

Total time for dyno calculation taken: 0.106050 seconds
Time taken per image for dyno calculation: 0.005302 seconds
Shifted algorithm stats for rgb:
{'mean': [0.35970781281260583, 0.4653655055772522, 0.49396675467059087], 'std': [0.17775270614878358, 0.16966115086553138, 0.16806500877316496]}


In [27]:
stats = shifted_data_variance(trainAimgs[:10]+trainBimgs[:10], gray=True)
print('Shifted algorithm stats for grayscale:')
print(stats)

Total time for dyno calculation taken: 0.042314 seconds
Time taken per image for dyno calculation: 0.002116 seconds
Shifted algorithm stats for grayscale:
{'mean': [0.4370183112307423, 0.4370183112307423, 0.4370183112307423], 'std': [0.16660754845647935, 0.16660754845647935, 0.16660754845647935]}


In [21]:
1800/6

300.0

## Next section checks the dimensions of saved intermediate images from CycleGAN model.

Note: All images are three channels, even if it was supposed to be grayscale, the single channel is copied to three channels

In [ ]:
outImgPath = '/scratch/kx59/arid/scratch_checkpoints/BS16_2gpuGrayscale/web/images/epoch001_fake_B.png' #BS16_2gpuGrayscale #in1out3
fake_B_img = Image.open(outImgPath, 'r')
print(fake_B_img.format)
display(fake_B_img)
fake_B_img = np.array(fake_B_img)
print(fake_B_img.shape)

In [ ]:
print(fake_B_img[0,0,:])

In [ ]:
real_A_ImgPath = '/scratch/kx59/arid/scratch_checkpoints/BS16_2gpuGrayscale/web/images/epoch001_real_A.png'
real_A_img = Image.open(real_A_ImgPath, 'r')
print(real_A_img.format)
display(real_A_img)
real_A_img = np.array(real_A_img)
print(real_A_img.shape)

In [ ]:
print(real_A_img[0,0,:])
truth_img = np.zeros_like(real_A_img[:,:,0])

test1 = (real_A_img[:,:,0]==real_A_img[:,:,1]).all() and (real_A_img[:,:,1]==real_A_img[:,:,2]).all()
print('Greyscale? %r' % test1)

In [ ]:
a = np.array([[1,2]])
b = np.array([[1,2]])
c = np.array([[1,2]])
print(a.shape, '\n', a)
print(b.shape, '\n', b)
print(c.shape, '\n', c)
test2 = (a==c).all() and (b==c).all()

print(test2)

## Below is experimenting with json file loading of norm stats

In [ ]:

import pickle

# obj0, obj1, obj2 are created here...
mean = (0.6, 0.6, 0.6)
std = (0.5, 0.5, 0.5)

# Saving the objects:
with open('stats.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([mean, std], f)
    
with open('stats.pkl', 'ab') as f:  # Python 3: open(..., 'wb')
    pickle.dump([mean, std], f)

del mean, std
try:
    print(mean)
except NameError as e:
    print(e)
try:
    print(std)
except NameError:
    print(f'Std not defined: {NameError}')

# Getting back the objects:
with open('stats.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    mean, std = pickle.load(f)
    
print(f'mean = {mean}', f'std = {std}', sep = '\n')

In [ ]:
import json
import torchvision.transforms as transforms
import numpy as np

# obj0, obj1, obj2 are created here...
mean = (0.6, 0.6, 0.6)
std = (0.5, 0.5, 0.5)

# Saving the objects:
with open('stats.json', 'w') as f:  # Python 3: open(..., 'wb')
    json.dump([mean, std], f)
    
with open('stats.pkl', 'a') as f:  # Python 3: open(..., 'wb')
    json.dump([mean, std], f)

del mean, std
try:
    print(mean)
except NameError as e:
    print(e)
try:
    print(std)
except NameError:
    print(f'Std not defined: {NameError}')

# Getting back the objects:
with open('stats.json', 'r') as f:  # Python 3: open(..., 'rb')
    mean, std = json.load(f)
    
print(f'mean = {mean}', f'std = {std}', sep = '\n')

norm = transforms.Normalize(mean,std)

In [3]:
# obj0, obj1, obj2 are created here...
# mean = np.array([0.6,0.6,0.6])
# std = np.array([0.5,0.5,0.5])
# import json
# import torchvision.transforms as transforms
mean = [0.6,0.6,0.6]
std = [0.5,0.5,0.5]
stats = {'Mean':mean, 'Std':std}

# Saving the objects:
with open('stats.json', 'w') as f:
    json.dump(stats, f)

del stats,mean, std
try:
    print(mean)
except NameError as e:
    print(e)
try:
    print(std)
except NameError as e:
    print(e)

# Getting back the objects:
with open('stats.json', 'r') as f:
    stats = json.load(f)
    
print(stats)
mean = stats['Mean']
std = stats['Std']
print(f'mean = {mean}', f'std = {std}', sep = '\n')
norm = transforms.Normalize(mean,std)
print(norm)

name 'mean' is not defined
name 'std' is not defined
{'Mean': [0.6, 0.6, 0.6], 'Std': [0.5, 0.5, 0.5]}
mean = [0.6, 0.6, 0.6]
std = [0.5, 0.5, 0.5]
Normalize(mean=[0.6, 0.6, 0.6], std=[0.5, 0.5, 0.5])


In [16]:
a = 1
b = 0
res = True if not a and b else False
print(res)

a = 1
b = 1
res = True if not a and b else False
print(res)

a = 0
btransforms
res = True if not a and b else False
print(res)

a = 0
b = 1
res = True if not a and b else False
print(res)

a = 1
b = 0
res = True if (not a) and b else False
print(res)

a = 1
b = 1
res = True if (not a) and b else False
print(res)

a = 0
b = 0
res = True if (not a) and b else False
print(res)

a = 0
b = 1
res = True if (not a) and b else False
print(res)

False
False
False
True
False
False
False
True


In [21]:
b = {'a':1,'b':2}
a = b
print(a)
print(b)
a['c']=3
print(a)
print(b)
b['d']=4
print(a)
print(b)

{'a': 1, 'b': 2}
{'a': 1, 'b': 2}
{'a': 1, 'b': 2, 'c': 3}
{'a': 1, 'b': 2, 'c': 3}
{'a': 1, 'b': 2, 'c': 3, 'd': 4}
{'a': 1, 'b': 2, 'c': 3, 'd': 4}


In [35]:
# import os
# cwd = os.getcwd()
# file = os.path.join(cwd,'test.json')
# print(file)
# with open(file,'w') as f:
#     json.dump(a,f)

/fs04/kx59/pytorch-CycleGAN-and-pix2pix/data/test.json


NameError: name 'json' is not defined